In [ ]:
import numpy as np
import cupy as cp
from Bio import SeqIO
import os, datetime
import matplotlib.pyplot as plt
%matplotlib inline

AA = "ARNDCQEGHILKMFPSTWYV*"
#FP_AAvsAA_MAP='./WAC.csv'
FP_AAvsAA_MAP='./BLOSUM62.csv'
LibraryType='./codon_num.csv'
uniprot = "input_uniprot\Protein_library\Human_reviewed"
output = 'output_path'


In [ ]:
#import library codon type
nnk = np.loadtxt(LibraryType, delimiter=',', dtype=np.str)
nnk = nnk[4,1:].astype(np.float)
nnk /= nnk.sum()

In [ ]:
#import score map 
AAvsAA_MAP = np.zeros((21,21), dtype=np.int)
AAvsAA_MAP[:-1,:-1] = np.loadtxt(FP_AAvsAA_MAP, delimiter=',', dtype=np.int)
AAvsAA_MAP[AAvsAA_MAP < 0] = 0
L=np.min(AAvsAA_MAP)
H = np.max(AAvsAA_MAP)

In [ ]:
def get_atos(codons: np.array, matrix: np.array) -> np.array:
    # Input
    # codons: np.array - Occurrence frequency of each amino acid based on codon_num.csb
    # matrix: np.array - Similarity table for amino acids
    # ------------------------------------------------------
    # Output
    # atos: np.array - A matrix where the amino acid types are along axis=0, and the probability distribution of scores for each amino acid is along axis=1
    atos = np.zeros((len(codons), H - L + 1))
    for i in range(len(codons)):
        for j in range(matrix.shape[0]):
            atos[i, matrix[i, j]] += codons[j]
    return atos

def get_atos_cupy(codons: cp.array, matrix: cp.array) -> cp.array:
    # Input
    # codons: cp.array - Occurrence frequency of each amino acid based on codon_num.csb
    # matrix: cp.array - Similarity table for amino acids
    # ------------------------------------------------------
    # Output
    # atos: cp.array - A matrix where the amino acid types are along axis=0, and the probability distribution of scores for each amino acid is along axis=1
    atos = cp.zeros((len(codons), H - L + 1))
    for i in range(len(codons)):
        for j in range(matrix.shape[0]):
            atos[i, matrix[i, j]] += codons[j]
    return atos

def fasta_to_index(file_name: str) -> np.array:
    # Input
    # file_name: str - Path to the fasta file
    # Output
    # out: list - Converts the peptides in the fasta file into amino acid indices based on AA
    out = []
    with open(file_name, "r") as f:
        for seq_record in SeqIO.parse(f, "fasta"):
            try:
                # print(seq_record.description)
                ind = np.array([AA.index(s) for s in list(seq_record.seq)], dtype=np.int8)
                out.append((seq_record.id, ind, seq_record.description))
            except ValueError:
                print("Invalid character found in {}".format(seq_record.id))
                continue
    print(out)
    return out

def fasta_to_index_cupy(file_name: str) -> cp.array:
    # Input
    # file_name: str - Path to the fasta file
    # Output
    # out: list - Converts the peptides in the fasta file into amino acid indices based on AA
    out = []
    with open(file_name, "r") as f:
        for seq_record in SeqIO.parse(f, "fasta"):
            try:
                # print(seq_record.description)
                ind = cp.array([AA.index(s) for s in list(seq_record.seq)], dtype=cp.int8)
                out.append((seq_record.id, ind, seq_record.description))
            except ValueError:
                print("Invalid character found in {}".format(seq_record.id))
                continue
    print(out)
    return out

def prep_numpy(uniprot, atos, output):
    st = datetime.datetime.now()
    print('Start:', st)
    num = 0
    Pmap = []
    # Create output directory
    if not os.path.exists(output):
        os.mkdir(output)
    for file in os.listdir(uniprot):
        # Read fasta
        try:
            Index = fasta_to_index('{}/{}'.format(uniprot, file))
        # If a character not included in AA is found, raise an error
        except ValueError:
            print("Invalid character found in {}".format(file))
            continue    

        for n in Index:
            if (num % 100 == 0):
                print(num)
            num += 1
            try:
                # Initialize the matrix to be saved in the CSV
                out = np.zeros((len(n[1]) - 11, (atos.shape[1] - 1) * 12 + 1))
            except ValueError:
                # Raise an error if the length is less than 12
                print("Peptide length of {} is too short".format(file))
                continue
            for i in range(len(n[1]) - 11):
                current_atos = atos[n[1][i:i + 12]]
                current_score = 1
                for j in range(current_atos.shape[0]):
                    current_score = np.convolve(current_score, current_atos[j])
                out[i] = current_score
            # print(out)
            Pmap.append((n[0], out))
    for n in range(len(Pmap)):
        np.savetxt(output + "/{:0=6}.csv".format(n), Pmap[n][1], fmt='%f', delimiter=',')

    et = datetime.datetime.now()
    print('End:', et)
    print('Time:', et - st)


In [ ]:
atos = get_atos(nnk, AAvsAA_MAP)
atos_c = get_atos_cupy(nnk, AAvsAA_MAP)

In [ ]:
prep_numpy(uniprot, atos, output)